In [ ]:
#In this example we will fetch the job description by pasting the URL,then we upload CV. Only then ChatGPT will
#analyze CV against the fetched job description. If the CV is a good match then it will write a cover letter.

#If 
    ##job posting url is fake/random text or 
    ##job posting is fake/random tex or 
    ##CV is fake/random text
#then ChatGPT will not analyze CV, it will give a generic response to enter the info correctly.

In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
import os
import io
import time
import requests
import PyPDF2
from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI
from ipywidgets import Textarea, FileUpload, Button, VBox, HTML

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [4]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [5]:
# Step 1: Create your prompts

system_prompt = "You are an assistant who analyzes user's CV against the job description \
 and provide a short summary if the user is fit for this job. If the user is fit for the job \
 write a cover letter for the user to apply for the job. Keep the cover letter professional, short, \
 and formal. \
 Important things to notice before analyzing CV:\
 1. Always check if the CV is actually a CV or just random text\
 2. Check if the job description fetched from the website is the job description or not\
     and ignore text related to navigation\
 3. Also check the link of the job posting, if it actually resembles a job posting or is just random \
     fake website\
 4. if any one of these two checks fails, do not analyze the CV against the Job description and give an\
     appropriate response as you think\
 5. Always respond in Markdown."

In [6]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [7]:
for_user_prompt = {
    'job_posting_url':'',
    'job_posting': '',
    'cv_text': ''
}

In [8]:
# Create widgets - to create a box for the job posting text
job_posting_url_area = Textarea(
    placeholder='Paste the URL of the job posting here, ONLY URL PLEASE',
    description='Fetching job:',
    disabled=False,
    layout={'width': '800px', 'height': '50px'}
)

status_job_posting = HTML(value="<b>Status:</b> Waiting for inputs...")

# Create Submit Buttons
fetch_job_posting_button = Button(description='Fetch Job Posting', button_style='primary')

def fetch_job_posting_action(b):
    for_user_prompt['job_posting_url'] = job_posting_url_area.value
    if for_user_prompt['job_posting_url']:
        ed = Website(for_user_prompt['job_posting_url'])
        status_job_posting.value = "<b>Status:</b> Job posting fetched successfully!"
        fetch_job_posting_button.button_style='success'
        for_user_prompt['job_posting']=ed.text
    else:
        status_job_posting.value = "<b>Status:</b> Please enter a job posting url before submitting."

# Attach actions to buttons
fetch_job_posting_button.on_click(fetch_job_posting_action)

# Layout
job_posting_box = VBox([job_posting_url_area, fetch_job_posting_button])

# Display all widgets
display(VBox([
    HTML(value="<h2>Input Job Posting Url</h2>"),
    job_posting_box,
    status_job_posting
]))

In [9]:
# Print fetched job description

print(for_user_prompt['job_posting'])

Skip to content
Search Jobs Home
Global Careers
Experian Life
Tech Careers
Early Careers
Locations and Teams
EMEA APAC
North America
Latin America
UK & Ireland
Saved jobs
0
Hamburger Menu
Search Jobs Home
Global Careers
Experian Life
Tech Careers
Early Careers
Locations and Teams
EMEA APAC
North America
Latin America
UK & Ireland
Staff Software Engineer, Data & AI Platform Architecture
__vacancyopjusttionswidget.opt-Location __
United States
__vacancyopjusttionswidget.opt-Department__
Product Development
__vacancyopjusttionswidget.opt-Employment__
Permanent
__vacancyopjusttionswidget.opt-Role Type__
Hybrid
__vacancyopjusttionswidget.opt-Schedule__
Full Time
__vacancyopjusttionswidget.opt-Salary Range__
$112,375 - $202,276
Apply
Shortlist
Send similar jobs by email
Share
Return to jobs page
We are looking for a Staff Software Engineer with a platform mindset to help shape the future of our enterprise-wide Data, Analytics, and AI/ML platform capabilities.
This role is not focused on buil

In [10]:
# Define file upload for CV
cv_upload = FileUpload(
    accept='.pdf',  # Only accept PDF files
    multiple=False,  # Only allow single file selection
    description='Upload CV (PDF)'
)

status = HTML(value="<b>Status:</b> Waiting for inputs...")

# Create Submit Buttons
submit_cv_button = Button(description='Submit CV', button_style='success')

# Functions
def submit_cv_action(change):

    if not for_user_prompt['cv_text']:
        status.value = "<b>Status:</b> Please upload a CV before submitting."
        
    if cv_upload.value:
        # Get the uploaded file
        uploaded_file = cv_upload.value[0]
        content = io.BytesIO(uploaded_file['content'])
        
        try:
            pdf_reader = PyPDF2.PdfReader(content) 
            cv_text = ""
            for page in pdf_reader.pages: 
                cv_text += page.extract_text() 
            
            # Store CV text in for_user_prompt
            for_user_prompt['cv_text'] = cv_text
            status.value = "<b>Status:</b> CV uploaded and processed successfully!"
        except Exception as e:
            status.value = f"<b>Status:</b> Error processing PDF: {str(e)}"

        time.sleep(0.5) # Short pause between upload and submit messages to display both
        
        if for_user_prompt['cv_text']:
            #print("CV Submitted:")
            #print(for_user_prompt['cv_text'])
            status.value = "<b>Status:</b> CV submitted successfully!"
            

# Attach actions to buttons
submit_cv_button.on_click(submit_cv_action)

# Layout
cv_buttons = VBox([submit_cv_button])

# Display all widgets
display(VBox([
    HTML(value="<h2>Import CV and submit</h2>"),
    cv_upload,
    cv_buttons,
    status
]))

In [11]:
# Prepare the user prompt that we will send to open ai (added URL for the context)
user_prompt = f"""
Job Posting: 
{for_user_prompt['job_posting']}

CV: 
{for_user_prompt['cv_text']}

Url:
{for_user_prompt['job_posting_url']}
"""

In [12]:
# Define messages with system_prompt and user_prompt
def messages_for(system_prompt_input, user_prompt_input):
    return [
        {"role": "system", "content": system_prompt_input},
        {"role": "user", "content": user_prompt_input}
    ]

In [13]:
# And now: call the OpenAI API. 
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages_for(system_prompt, user_prompt)
)

# Response is provided in Markdown and displayed accordingly
display(Markdown(response.choices[0].message.content))

Based on the job description provided and the CV of Kevin Bardool, it appears that Kevin is a strong fit for the position of Staff Software Engineer for Data & AI Platform Architecture at Experian.

### Fit Summary:
- **Experience**: Kevin has over 20 years of software engineering experience, with roles that involved significant data applications and architecture, aligning closely with the requirements for building foundational capabilities for data-driven and AI-enabled products.
- **Technical Skills**: Proficiency in programming languages such as Python aligns with the job requirements. His experience with cloud platforms (AWS), databases, and modern data architectures aligns well with the needs for cloud-native architecture and big data processing frameworks.
- **Leadership and Collaboration**: His history of leading teams and projects at Experian shows he can influence technology adoption and drive platform reuse, which are crucial aspects of the new role.
- **Research and Development**: His Ph.D. work in machine learning and advanced research experience complements the role’s focus on driving innovations in AI/ML capabilities.

Given this match, here’s a cover letter for Kevin to apply for the job.

---

**Kevin Bardool**  
39 Santa Monica  
Aliso Viejo, CA 92656  
kbardool@gmail.com  
949.373.5880  

[Date]

**Hiring Manager**  
Experian  
[Company Address]  
United States  

Dear Hiring Manager,

I am writing to express my interest in the Staff Software Engineer position for Data & AI Platform Architecture at Experian, as advertised on your careers page. With over 20 years of experience in software engineering, including a significant focus on data and machine learning applications, I am excited about the opportunity to contribute to Experian's innovative platform capabilities.

My background includes extensive work in developing scalable and secure data systems during my tenure at Experian Information Solutions, where I played a crucial role in cloud migrations and database management. I am proficient in Python and have hands-on experience with AWS and modern data architectures, positioning me well to meet the technical requirements of this role. 

In addition to my technical skills, I am passionate about collaboration and leadership. My experience leading cross-functional teams in research and practical implementations of cutting-edge technologies such as federated learning and AI-driven data solutions aligns closely with Experian's vision for data-driven products. 

I am eager to bring my unique blend of expertise in software engineering, data science, and machine learning to Experian, and I am excited about the possibilities of shaping the future of your enterprise-wide data and AI platform capabilities.

Thank you for considering my application. I look forward to the opportunity to discuss how my experience and vision align with the goals of Experian.

Sincerely,

Kevin Bardool

--- 

Feel free to adjust any details as needed before sending!